# Process Observation Data Analysis
## The aim of this analysis is to experiment with an alternative approach to analyzing process data that could potentially be more time efficient.

The data structure for the data can be seen in the image below. The observations performed vary in the level of completness as some have been observed more than others, while some have not been observed at all. And for some the sample size is large enough and variance low enough to give confidence that an average is representative of the sample.

![ERD Image](../Data/ERD.jpg)

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from IPython.display import display
import seaborn as sns

In [4]:
data_folder = '../Data'
full_path = os.path.join(data_folder, 'ProcessData.xlsx')
excel_file_obj = pd.ExcelFile(full_path)
print('The names of the tabs or sheets in the excel file are as follows: ' ,excel_file_obj.sheet_names)

The names of the tabs or sheets in the excel file are as follows:  ['Iteration 1 Data (2)', 'Iteration 1 Data', 'Iteration 2 Tasks', 'Iteration 2 Session', 'DurationData', 'TaskData', 'RecordingData', 'ProcessData', 'HourlyRates']


From the list of sheets above it is clear that there are some additional unwanted sheets. Upon investigation it looks like the other sheets are templates that were imporoperly deleted and out of best practice we will not be dedicating resources to parsing them.

In [10]:
sheets_to_import = ['DurationData', 'TaskData', 'RecordingData', 'ProcessData', 'HourlyRates']
select_sheets = pd.read_excel(full_path, sheet_name=sheets_to_import)

In [12]:
display(select_sheets['DurationData'].sample(n=10))

DurationID  RecordingID      TaskID StartTime   EndTime        Duration
431         433          130    413100.0  00:54:24  00:55:15 0 days 00:00:51
517         519           30   5371200.0  01:42:17  01:43:36 0 days 00:01:19
198         199           90   5123300.0  00:37:44  00:41:00 0 days 00:03:16
109         110           61   5121100.0  03:02:33  03:02:43 0 days 00:00:10
105         106           61   5121500.0  01:40:15  01:44:39 0 days 00:04:24
309         310           80   5322200.0  02:35:00  02:39:18 0 days 00:04:18
122         123           51    511300.0  01:05:28  01:06:30 0 days 00:01:02
524         526           30  53841600.0  03:00:42  03:02:36 0 days 00:01:54
417         419          130    413400.0  00:15:01  00:19:52 0 days 00:04:51
222         223           90   5121500.0  03:37:05  03:37:26 0 days 00:00:21

For the DurationData table:
1. The TaskID needs to be fixed and the .0 needs to be removed
2. The Duration needs to be fixed, and the days need to be removed, or recalculate the Duration

In [14]:
display(select_sheets['TaskData'].sample(n=10))

TaskID                                           TaskName  ProcessID  \
130   5216500  Check service providers & add practice discipl...        521   
151  52231300                   Check RPL codes on  Medcode link        522   
118  52131100  Generate pre-authorisation confirmation & auth...        521   
170   4311100                         Generate prepay run report       4311   
76     415500                                      Process claim        415   
90      45100                                      Email reports         45   
73     415200                                      Capture claim        415   
50     539100                                        Review case        539   
39     411300                                       Click Assess        411   
183   5382200  Assess case for clinical appropriateness for e...       5382   

              ResponsibleRole  Iterations AverageTaskTime  
130              Case manager           0             NaN  
151   Pre-authorization Agent           0             NaN  
118   Pre-authorization Agent           0             NaN  
170  BSS Technical Specialist           0             NaN  
76            Claims Assessor           0             NaN  
90                    BI Team           0             NaN  
73            Claims Assessor           0             NaN  
50               Case manager           0             NaN  
39            Claims Assessor           0             NaN  
183              Case manager           3        00:00:00

For the TaskData table:
1. We need to know up front how many processes we actually observed, then remove all processes that were not observed at least once
2. We need to know the number of iterations observed for each task, as well as how many people were observed. (Group and individualized views)
3. We need all measures of central tendency and either a box plot or scatter plot to get an idea of how times are distributed for each TaskID.
4. Find a relationship between number of iterations or people observed, and how data is distributed (looking at the mean as a proportion of the standard deviation)

In [20]:
display(select_sheets['RecordingData'])

RecordingID  Recording Length RecordingDate     PersonObserved  \
0               10.0          00:46:42    28/08/2024    Pauline Jordaan   
1               11.0          00:29:04    28/08/2024    Pauline Jordaan   
2               20.0          03:21:07    28/08/2024  Siyethemba Mkhize   
3               30.0          03:03:36    29/08/2024  Jaqueline Wessels   
4               40.0          00:42:28    02/09/2024        Zoleka Baba   
...              ...               ...           ...                ...   
1048570          NaN               NaN           NaN                NaN   
1048571          NaN               NaN           NaN                NaN   
1048572          NaN               NaN           NaN                NaN   
1048573          NaN               NaN           NaN                NaN   
1048574          NaN  2 days, 17:25:46           NaN                NaN   

                RecordingStatus  
0        Processed and Reviewed  
1        Processed and Reviewed  
2        Processed and Reviewed  
3        Processed and Reviewed  
4        Processed and Reviewed  
...                         ...  
1048570                     NaN  
1048571                     NaN  
1048572                     NaN  
1048573                     NaN  
1048574                     NaN  

[1048575 rows x 5 columns]

For the RecordingData table:
1. The data type for Recording ID needs to be fixed
2. The extra rows need to be removed